# NBA Player Performance - Exploratory Data Analysis (EDA)

## Objective
The purpose of this notebook is to explore and prepare NBA play_by_play data for downstream analysis.
Specifically, we focus on validating data integrity and deriving a clean, player-game level scoring dataset 
from raw event-level records.

This notebook emphasizes **data understanding, transformation, and validation**, rather than insights generation storytelling.

---

## Data source

    
**Dataset:** NBA Dataset (Kaggle)

**Storage:** SQLite ('nba.sqlite')

**Granularity:** Event-level play-by-play records across multiple NBA seasons 

Raw player box-score totals are not directly available and must be derived from cumulative scoring information embedded in play-by-play text descriptions. 

---

## Scope of EDA

This ecploratory analysis includes:

    -Database schema inspection and table validation.
    -Identification of relevant entities (players,games, events).
    -Feature engineering tp derive final points scored per player per game.
    -Data quality checks ( uniqueness, missing values, distribution sanity)
    -Export of a validated, analysis-ready dataset.

Insights, comparisons, and visual storytelling are intentionally defered to a separate analysis notebook.

>**Note:** This notebook prioritizes correctness and reproducibility over completness.
>Intermediate checks and validations are included to ensure reliability of derived metrics.


In [1]:
import sqlite3
import pandas as pd 

DB_PATH="../data/raw/nba.sqlite"

conn=sqlite3.connect(DB_PATH)
print("Connected to database:", DB_PATH)

Connected to database: ../data/raw/nba.sqlite


In [2]:
tables = pd.read_sql("""
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
""",conn)

tables

,name
0,common_player_info
1,draft_combine_stats
2,draft_history
3,game
4,game_info
5,game_summary
6,inactive_players
7,line_score
8,officials
9,other_stats


In [3]:
print(f"Number of tables in database: {len(tables)}")

Number of tables in database: 16


In [4]:
df_player = pd.read_sql(
    "SELECT * FROM player LIMIT 10;",
    conn
)
df_player

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,0
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,0
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,0
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,0
5,949,Shareef Abdur-Rahim,Shareef,Abdur-Rahim,0
6,76005,Tom Abernethy,Tom,Abernethy,0
7,76006,Forest Able,Forest,Able,0
8,76007,John Abramovic,John,Abramovic,0
9,203518,Alex Abrines,Alex,Abrines,0


In [5]:
pd.read_sql("PRAGMA table_info(player);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,full_name,TEXT,0,None,0
2,2,first_name,TEXT,0,None,0
3,3,last_name,TEXT,0,None,0
4,4,is_active,INTEGER,0,None,0


In [6]:
df_stats=pd.read_sql(
    "SELECT * FROM other_stats LIMIT 10;",
    conn)
df_stats

,game_id,league_id,team_id_home,team_abbreviation_home,team_city_home,pts_paint_home,pts_2nd_chance_home,pts_fb_home,largest_lead_home,lead_changes,...,team_abbreviation_away,team_city_away,pts_paint_away,pts_2nd_chance_away,pts_fb_away,largest_lead_away,team_turnovers_away,total_turnovers_away,team_rebounds_away,pts_off_to_away
0,0029600012,00,1610612756,PHX,Phoenix,44,18,2,1,4,...,LAL,Los Angeles,42,10,13,19,0,23,11,None
1,0029600005,00,1610612737,ATL,Atlanta,32,9,6,0,0,...,MIA,Miami,32,15,14,16,1,19,6,None
2,0029600002,00,1610612739,CLE,Cleveland,36,14,6,20,1,...,NJN,New Jersey,26,16,4,2,1,22,12,None
3,0029600007,00,1610612754,IND,Indiana,34,11,4,10,7,...,DET,Detroit,30,14,7,9,2,19,10,None
4,0029600013,00,1610612746,LAC,Los Angeles,40,19,2,12,5,...,GSW,Golden State,30,9,2,6,0,20,7,None
5,0029600009,00,1610612745,HOU,Houston,44,16,20,25,8,...,SAC,Sacramento,38,9,12,2,0,27,11,None
6,0029600014,00,1610612757,POR,Portland,62,24,19,35,3,...,VAN,Vancouver,34,8,6,2,1,16,9,None
7,0029600001,00,1610612738,BOS,Boston,48,18,15,16,8,...,CHI,Chicago,56,12,15,11,0,19,13,None
8,0029600006,00,1610612761,TOR,Toronto,28,11,14,7,16,...,NYK,New York,38,10,15,11,0,24,13,None
9,0029600003,00,1610612749,MIL,Milwaukee,46,27,9,14,4,...,PHI,Philadelphia,54,8,24,2,1,14,10,None


In [7]:
df_pbp= pd.read_sql(
    "SELECT * FROM play_by_play LIMIT 10;",
    conn
)
df_pbp

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,0029600012,0,12,0,1,14:43 PM,12:00,None,Start of 1st Period (14:43 PM EST),None,...,None,None,0.0,0,None,None,None,None,None,0
1,0029600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,None,None,...,Suns,PHX,5.0,208,Sam Cassell,1610612756.0,Phoenix,Suns,PHX,0
2,0029600012,3,2,1,1,14:51 PM,11:45,None,None,MISS Cassell 15' Jump Shot,...,None,None,0.0,0,None,None,None,None,None,0
3,0029600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),None,None,...,None,None,0.0,0,None,None,None,None,None,0
4,0029600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,None,None,...,None,None,0.0,0,None,None,None,None,None,0
5,0029600012,6,4,0,1,14:51 PM,11:27,None,None,Cassell REBOUND (Off:0 Def:1),...,None,None,0.0,0,None,None,None,None,None,0
6,0029600012,7,6,1,1,14:51 PM,11:14,Van Exel P.FOUL (P1.T1),None,None,...,None,None,0.0,0,None,None,None,None,None,0
7,0029600012,8,5,1,1,14:52 PM,11:08,None,None,Cassell Bad Pass Turnover (P1.T1),...,None,None,0.0,0,None,None,None,None,None,0
8,0029600012,9,2,5,1,14:52 PM,10:49,MISS Ceballos 1' Layup,None,Horry BLOCK (1 BLK),...,None,None,5.0,109,Robert Horry,1610612756.0,Phoenix,Suns,PHX,0
9,0029600012,10,4,0,1,14:52 PM,10:49,LAKERS Rebound,None,None,...,None,None,0.0,0,None,None,None,None,None,0


In [8]:
pd.read_sql("PRAGMA table_info(play_by_play);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,eventnum,INTEGER,0,None,0
2,2,eventmsgtype,INTEGER,0,None,0
3,3,eventmsgactiontype,INTEGER,0,None,0
4,4,period,INTEGER,0,None,0
5,5,wctimestring,TEXT,0,None,0
6,6,pctimestring,TEXT,0,None,0
7,7,homedescription,TEXT,0,None,0
8,8,neutraldescription,TEXT,0,None,0
9,9,visitordescription,TEXT,0,None,0


In [9]:
sample_scoring = pd.read_sql("""
SELECT
    game_id,
    player1_id,
    homedescription,
    visitordescription,
    neutraldescription
FROM play_by_play
WHERE homedescription LIKE '%PTS%'
    OR visitordescription LIKE '%PTS%'
    OR homedescription LIKE '%3PTS%'
    OR visitordescription LIKE '%3PTS%'
LIMIT 20;
""", conn)

sample_scoring

,game_id,player1_id,homedescription,visitordescription,neutraldescription
0,0029600012,406,O'Neal Slam Dunk (2 PTS) (Van Exel 1 AST),None,None
1,0029600012,76,Ceballos Layup (2 PTS) (Jones 1 AST),None,None
2,0029600012,208,None,Cassell Free Throw 1 of 2 (1 PTS),None
3,0029600012,208,None,Cassell Free Throw 2 of 2 (2 PTS),None
4,0029600012,445,None,Person Layup (2 PTS) (Kleine 1 AST),None
5,0029600012,922,Campbell Free Throw 1 of 2 (1 PTS),None,None
6,0029600012,406,O'Neal 7' Hook Shot (4 PTS) (Jones 2 AST),None,None
7,0029600012,76,Ceballos Layup (4 PTS) (Fisher 1 AST),None,None
8,0029600012,445,None,Person Free Throw 1 of 2 (3 PTS),None
9,0029600012,445,None,Person Free Throw 2 of 2 (4 PTS),None


In [10]:
import re
query="""
SELECT 
    game_id,
    player1_id,
    COALESCE(homedescription, visitordescription) AS description
FROM play_by_play
WHERE description LIKE '%PTS%'
"""

df_scoring=pd.read_sql(query, conn)

df_scoring.head()

,game_id,player1_id,description
0,0029600012,406,O'Neal Slam Dunk (2 PTS) (Van Exel 1 AST)
1,0029600012,76,Ceballos Layup (2 PTS) (Jones 1 AST)
2,0029600012,208,Cassell Free Throw 1 of 2 (1 PTS)
3,0029600012,208,Cassell Free Throw 2 of 2 (2 PTS)
4,0029600012,445,Person Layup (2 PTS) (Kleine 1 AST)


In [11]:
df_scoring["cumulative_pts"]=(
    df_scoring["description"]
    .str.extract(r"\((\d+)\sPTS\)")
    .astype(int)
)

df_scoring.head()

,game_id,player1_id,description,cumulative_pts
0,0029600012,406,O'Neal Slam Dunk (2 PTS) (Van Exel 1 AST),2
1,0029600012,76,Ceballos Layup (2 PTS) (Jones 1 AST),2
2,0029600012,208,Cassell Free Throw 1 of 2 (1 PTS),1
3,0029600012,208,Cassell Free Throw 2 of 2 (2 PTS),2
4,0029600012,445,Person Layup (2 PTS) (Kleine 1 AST),2


In [12]:
player_game_points=(
    df_scoring
    .groupby(["game_id","player1_id"], as_index=False)
    .agg(final_points=("cumulative_pts","max"))
)
player_game_points

,game_id,player1_id,final_points
0,0011300001,200757,8
1,0011300001,201142,24
2,0011300001,201586,15
3,0011300001,201934,4
4,0011300001,202704,9
...,...,...,...
545021,0049800087,764,15
545022,0049800087,84,35
545023,0049800087,913,7
545024,0049800087,948,7


In [13]:
df_players = pd.read_sql("""
    SELECT 
        id AS player_id,
        full_name
    FROM player;
    """, conn)

df_players.head()

,player_id,full_name
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad


In [14]:
player_game_points_named = (
    player_game_points
    .merge(
        df_players,
        left_on="player1_id",
        right_on="player_id",
        how="left"
    )
    .drop(columns=["player_id"])
)
player_game_points_named

,game_id,player1_id,final_points,full_name
0,0011300001,200757,8,Thabo Sefolosha
1,0011300001,201142,24,Kevin Durant
2,0011300001,201586,15,Serge Ibaka
3,0011300001,201934,4,Hasheem Thabeet
4,0011300001,202704,9,Reggie Jackson
...,...,...,...,...
545021,0049800087,764,15,David Robinson
545022,0049800087,84,35,Latrell Sprewell
545023,0049800087,913,7,Larry Johnson
545024,0049800087,948,7,Marcus Camby


In [15]:
player_game_points_named.sort_values(
    by="final_points",
    ascending= False
).head(10)

,game_id,player1_id,final_points,full_name
117612,0020500591,977,81,Kobe Bryant
440781,0022200917,203081,71,Damian Lillard
434622,0022200552,1628378,71,Donovan Mitchell
142592,0020600977,977,65,Kobe Bryant
251443,0021300640,2546,62,Carmelo Anthony
114107,0020500359,977,62,Kobe Bryant
85062,0020300927,1503,62,Tracy McGrady
378799,0021900652,203081,61,Damian Lillard
508941,0029900856,406,61,Shaquille O'Neal
385330,0021901300,203081,61,Damian Lillard


## Data Quality Checks 

Before proceeding to analisys, we validate the derived player-game dataset
to ensure structural and logical consistency

In [16]:
#Each row should represent exactly one player-game
player_game_points_named.duplicated(
    subset=["game_id","player1_id"]
).sum()

0

In [17]:
player_game_points_named.isna().mean().sort_values(ascending=False)

full_name       0.001404
game_id         0.000000
player1_id      0.000000
final_points    0.000000
dtype: float64

- A small fraction of player records (<0.2%) could not be matched to player names due to historical data gaps; these cases do not impact metric validity.


In [18]:
player_game_points_named["final_points"].describe()

count    545026.000000
mean         11.061089
std           7.817991
min           1.000000
25%           5.000000
50%           9.000000
75%          15.000000
max          81.000000
Name: final_points, dtype: float64

In [19]:
player_game_points_named["final_points"].quantile([0.01,0.99])

0.01     1.0
0.99    35.0
Name: final_points, dtype: float64

In [20]:
player_game_points_named[
        player_game_points_named["final_points"]>=80
][["full_name","final_points","game_id"]].head()

,full_name,final_points,game_id
117612,Kobe Bryant,81,0020500591


In [21]:
player_game_points_named.to_csv(
    "../data/processed/player_game_points.csv",
    index=False
)

## Export game metadata

To keep downstream notebooks lightweight and reproducible, we export a small game-level metadata table 

In [25]:
df_games_meta=pd.read_sql("""
SELECT game_id, season_id,season_type, game_date
FROM game;
""", conn)
#Standardize game_id formating 
df_games_meta["game_id"]=df_games_meta["game_id"].astype(str).str.strip().str.zfill(10)

df_games_meta.to_csv("../data/processed/games_meta.csv",index=False)
df_games_meta.head()

,game_id,season_id,season_type,game_date
0,0024600001,21946,Regular Season,1946-11-01 00:00:00
1,0024600003,21946,Regular Season,1946-11-02 00:00:00
2,0024600002,21946,Regular Season,1946-11-02 00:00:00
3,0024600004,21946,Regular Season,1946-11-02 00:00:00
4,0024600005,21946,Regular Season,1946-11-02 00:00:00


## Derive rebound events per player-game

Similiar to scoring, rebound participation is derived from play-by-play text
to enable additional performance features for downstream analysis and modeling.

In [26]:
df_rebounds=pd.read_sql("""
SELECT
    game_id,
    player1_id
FROM play_by_play
WHERE homedescription LIKE '%REBOUND%'
    OR visitordescription LIKE '%REBOUND%';
""", conn)

df_rebounds["game_id"]=df_rebounds["game_id"].astype(str).str.zfill(10)
df_rebounds["player1_id"]=df_rebounds["player1_id"].astype(str)

df_rebounds_pg=(
    df_rebounds
        .groupby(["game_id","player1_id"])
        .size()
        .reset_index(name="rebound_events")
)

df_rebounds_pg.head()

,game_id,player1_id,rebound_events
0,0011300001,12321,9
1,0011300001,1610612760,10
2,0011300001,200757,2
3,0011300001,201142,8
4,0011300001,201586,6


In [29]:
player_game_points_named["game_id"]=(
    player_game_points_named["game_id"].astype(str).str.zfill(10)
)

player_game_points_named["player1_id"]=(
    player_game_points_named["player1_id"].astype(str)
)

player_game_features=(
    player_game_points_named
        .merge(
            df_rebounds_pg,
            on=["game_id","player1_id"],
            how="left"
        )
)

player_game_features["rebound_events"]=(
    player_game_features["rebound_events"].fillna(0)
)

player_game_features.head()

,game_id,player1_id,final_points,full_name,rebound_events
0,0011300001,200757,8,Thabo Sefolosha,2.0
1,0011300001,201142,24,Kevin Durant,8.0
2,0011300001,201586,15,Serge Ibaka,6.0
3,0011300001,201934,4,Hasheem Thabeet,5.0
4,0011300001,202704,9,Reggie Jackson,2.0


In [30]:
player_game_features.to_csv(
    "../data/processed/player_game_features.csv",
    index=False
)

### EDA Summary

    -Player scoring was successfuly derived from raw play_by_play tex using cumulative point values.
    -The resulting dataset contains on row per player per game with no duplicate keys.
    -Scoring distributions and summary statistics fall within expect NBA ranges.
    -The processed dataset is suitable for downstream analytical and comparative analysis.


The cleaned and validated dataset is exported for use in the analysis notebook.
